# Inspektor Gadget Performance Benchmarks

This document provides an overview of the performance impact that Inspektor
Gadget has on the system. The document contains different experiments showing
different performance aspects.

## CPU and Memory Overhead 

This experiment measures the CPU and memory overhead introduced by Inspektor
Gadget (`ig` binary) when running on a host.

### Methodology 

The total total CPU and memory usage of the host is measured while an
application generatess a fixed amount of events per second. The experiment is
run with and without Inspektor Gadget enabled to compare the overhead. In this
scenario, a single gadget is run at the time with the `ig` binary.

The memory and CPU are only captured after the gadget is initialized and before
it's stopped, so it only accounts for steady state performance.

## Results

The following cells load and visualize the benchmark results.
The produced plots show:
- **Host Performance**: Comparison between baseline (without IG) and total system performance with IG enabled.
- **IG Process**: Specific overhead introduced by the `ig` process itself (user space only).


### Input File

Set the input file path to the CSV file generated by `make benchmarks-test`

In [ ]:
import os

# Configuration:
# Either set input_file variable or use the environment variable INPUT_FILE
#input_file = 'test_results_2025_...csv'
input_file = os.environ['INPUT_FILE']

# Configuration: Hide code cells when exporting to HTML
# Set to True to hide code cells, False to show them
hide_code_cells = True

## Machine Specifications

The following section shows the specifications of the machine used for the benchmarks, as recorded in the CSV file.

In [ ]:
def extract_machine_specs(filepath):
    """Extract machine specifications from CSV comment lines"""
    specs = []
    try:
        with open(filepath, 'r') as file:
            for line in file:
                line = line.strip()
                if line.startswith('#'):
                    # Remove the # and any leading/trailing whitespace
                    spec_line = line[1:].strip()
                    if spec_line:  # Only add non-empty lines
                        specs.append(spec_line)
                else:
                    # Stop reading when we reach the actual CSV data
                    break
    except Exception as e:
        print(f"Error reading machine specs: {e}")

    return specs

# Extract and display machine specifications
if input_file and os.path.exists(input_file):
    machine_specs = extract_machine_specs(input_file)

    if machine_specs:
        print("Machine Specifications:")
        print("=" * 50)
        for spec in machine_specs:
            print(f"• {spec}")
        print("=" * 50)
    else:
        print("No machine specifications found in the CSV file.")
elif input_file:
    print(f"Error: Input file '{input_file}' not found for machine specs")
else:
    print("No input file specified for machine specs.")

In [ ]:
%matplotlib inline

# Import required libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob
from IPython.display import display, HTML
from scipy import stats

# Configure HTML output to hide code cells if requested
if 'hide_code_cells' in locals() and hide_code_cells:
    display(HTML('''
    <style>
        .jp-CodeCell .jp-Cell-inputWrapper {
            display: none !important;
        }
        .jp-CodeCell .jp-Cell-inputCollapser {
            display: none !important;
        }
        /* For classic notebook interface */
        div.input {
            display: none !important;
        }
        div.prompt {
            display: none !important;
        }
    </style>
    '''))

#from IPython.display import set_matplotlib_formats
#set_matplotlib_formats('png', 'pdf')

In [ ]:
def calculate_stats(data):
    """Calculate mean and 95% confidence interval for a dataset"""
    if len(data) == 0:
        return 0, 0

    mean_val = np.mean(data)
    if len(data) == 1:
        return mean_val, 0

    # Calculate 95% confidence interval using t-distribution
    sem = stats.sem(data)  # Standard error of the mean
    ci = sem * stats.t.ppf((1 + 0.95) / 2, len(data) - 1)

    return mean_val, ci

In [ ]:
def aggregate_data(df_gadget):
    """Aggregate data by test type and events per second, calculating means and CIs"""
    aggregated = []

    # Group by TestCase and EventsPerSecond
    for (test_name, eps), group in df_gadget.groupby(['TestCase', 'EventsPerSecond']):
        cpu_mean, cpu_ci = calculate_stats(group['Cpu'].values)
        mem_mean, mem_ci = calculate_stats(group['Mem'].values)
        ig_cpu_mean, ig_cpu_ci = calculate_stats(group['IgCpu'].values)
        ig_mem_mean, ig_mem_ci = calculate_stats(group['IgMem'].values)
        lost_mean, lost_ci = calculate_stats(group['Lost'].values)

        aggregated.append({
            'Name': test_name.strip(),
            'rps': eps,
            '%cpu': cpu_mean,
            'cpu_ci': cpu_ci,
            'mem(MB)': mem_mean,  # Assuming already in MB based on sample data
            'mem_ci': mem_ci,
            'ig_cpu_mean': ig_cpu_mean,
            'ig_cpu_ci': ig_cpu_ci,
            'ig_mem_mean': ig_mem_mean,  # Assuming already in MB based on sample data
            'ig_mem_ci': ig_mem_ci,
            'lost': lost_mean
        })

    return pd.DataFrame(aggregated)

In [ ]:
def create_performance_table(baseline_data, ig_data):
    """Create a formatted table with performance metrics"""
    rps_values = baseline_data['rps'].values
    baseline_cpu = baseline_data['%cpu'].values
    ig_cpu = ig_data['%cpu'].values
    baseline_mem = baseline_data['mem(MB)'].values
    ig_mem = ig_data['mem(MB)'].values
    ig_lost = ig_data['lost'].values

    # Get IG process specific data
    ig_process_cpu = ig_data['ig_cpu_mean'].values
    ig_process_mem = ig_data['ig_mem_mean'].values

    # Create table data
    table_data = {
        'RPS': rps_values,
        'Baseline CPU (%)': [f'{cpu:.2f}' for cpu in baseline_cpu],
        'With IG CPU (%)': [f'{cpu:.2f}' for cpu in ig_cpu],
        'IG Process CPU (%)': [f'{cpu:.2f}' for cpu in ig_process_cpu],
        'Baseline Mem (MB)': [f'{mem:.0f}' for mem in baseline_mem],
        'With IG Mem (MB)': [f'{mem:.0f}' for mem in ig_mem],
        'IG Process Mem (MB)': [f'{mem:.0f}' for mem in ig_process_mem],
        'Lost Events': [int(lost) for lost in ig_lost]
    }

    df_table = pd.DataFrame(table_data)

    # Style the table for better display
    styled_table = df_table.style.set_table_styles([
        {'selector': 'thead th', 'props': [('background-color', '#4CAF50'),
                                          ('color', 'white'),
                                          ('font-weight', 'bold'),
                                          ('text-align', 'center')]},
        {'selector': 'tbody td', 'props': [('text-align', 'center')]},
        {'selector': 'table', 'props': [('border-collapse', 'collapse'),
                                        ('margin', '25px 0'),
                                        ('font-size', '0.9em'),
                                        ('min-width', '400px'),
                                        ('border-radius', '5px 5px 0 0'),
                                        ('overflow', 'hidden'),
                                        ('box-shadow', '0 0 20px rgba(0,0,0,0.15)')]},
        {'selector': 'tbody tr', 'props': [('border-bottom', '1px solid #dddddd')]},
    ])

    return df_table, styled_table

In [ ]:
def create_ig_process_plots(baseline_data, ig_data, gadget):
    """Create plots showing the overhead introduced by the IG process itself"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

    # Get data for plotting
    rps_values = baseline_data['rps'].values
    ig_process_cpu = ig_data['ig_cpu_mean'].values
    ig_process_mem = ig_data['ig_mem_mean'].values
    ig_process_cpu_ci = ig_data['ig_cpu_ci'].values
    ig_process_mem_ci = ig_data['ig_mem_ci'].values

    x = np.arange(len(rps_values))

    # Plot 1: IG Process CPU Usage
    bars1 = ax1.bar(x, ig_process_cpu, alpha=0.8, color='orange',
                    yerr=ig_process_cpu_ci, capsize=5, label='IG Process CPU')

    ax1.set_xlabel('Events per Second')
    ax1.set_ylabel('CPU Usage (%)')
    ax1.set_title('CPU Usage (with 95% CI)')
    ax1.set_xticks(x)
    ax1.set_xticklabels(rps_values)
    ax1.grid(True, alpha=0.3, axis='y')

    # Set dynamic Y-axis range for CPU
    max_cpu = np.max(ig_process_cpu)
    ax1.set_ylim(0, max_cpu * 1.25)

    # Add value labels on CPU bars
    for i, bar in enumerate(bars1):
        height = bar.get_height()
        ci_value = ig_process_cpu_ci[i]
        ax1.text(bar.get_x() + bar.get_width()/2., height + ci_value,
                 f'{height:.2f}', ha='center', va='bottom', fontsize=9, rotation=0)

    # Plot 2: IG Process Memory Usage
    bars2 = ax2.bar(x, ig_process_mem, alpha=0.8, color='red',
                    yerr=ig_process_mem_ci, capsize=5, label='IG Process Memory')

    ax2.set_xlabel('Events per Second')
    ax2.set_ylabel('Memory Usage (MB)')
    ax2.set_title('Memory Usage (with 95% CI)')
    ax2.set_xticks(x)
    ax2.set_xticklabels(rps_values)
    ax2.grid(True, alpha=0.3, axis='y')

    # Set dynamic Y-axis range for memory
    max_mem = np.max(ig_process_mem)
    min_mem = np.min(ig_process_mem)
    if max_mem > 0:
        ax2.set_ylim(0.75*min_mem, 1.25*max_mem)
    else:
        ax2.set_ylim(0, 10)

    # Add value labels on Memory bars
    for i, bar in enumerate(bars2):
        height = bar.get_height()
        ci_value = ig_process_mem_ci[i]
        ax2.text(bar.get_x() + bar.get_width()/2., height + ci_value,
                 f'{height:.0f}', ha='center', va='bottom', fontsize=9, rotation=0)

    # Set main title
    fig.suptitle(f'{gadget} - IG Process', fontsize=16, fontweight='bold')

    plt.tight_layout()

    return fig

In [ ]:
def create_performance_plots(filepath):
    """Create CPU and memory performance plots for a single test result file"""
    # Read the CSV file, ignoring lines starting with #
    df = pd.read_csv(filepath, comment='#')

    # Clean column names (remove extra spaces)
    df.columns = df.columns.str.strip()

    # Get unique test names
    test_name = df['TestName'].unique()
    all_results = []

    for test in test_name:
        # Filter data for this gadget
        df_test = df[df['TestName'] == test].copy()

        # Aggregate the data (calculate means and CIs)
        aggregated_df = aggregate_data(df_test)

        # Separate baseline and ig data
        baseline_data = aggregated_df[aggregated_df['Name'] == 'baseline']
        ig_data = aggregated_df[aggregated_df['Name'] == 'ig']

        if len(baseline_data) == 0 or len(ig_data) == 0:
            print(f"Warning: Missing baseline or ig data for gadget {test}")
            continue

        # Set up the main figure
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

        # Get data for plotting
        rps_values = baseline_data['rps'].values
        baseline_cpu = baseline_data['%cpu'].values
        ig_cpu = ig_data['%cpu'].values
        baseline_cpu_ci = baseline_data['cpu_ci'].values
        ig_cpu_ci = ig_data['cpu_ci'].values

        # Plot 1: CPU Usage - simple comparison bars
        x = np.arange(len(rps_values))
        width = 0.35

        # Baseline bar
        bars1 = ax1.bar(x - width/2, baseline_cpu, width, label='Baseline', alpha=0.8,
                        yerr=baseline_cpu_ci, capsize=5)

        # IG Total bar
        bars2 = ax1.bar(x + width/2, ig_cpu, width, label='With IG', alpha=0.8,
                        yerr=ig_cpu_ci, capsize=5)

        ax1.set_xlabel('Events per Second')
        ax1.set_ylabel('CPU Usage (%)')
        ax1.set_title('CPU Usage (with 95% CI)')
        ax1.set_xticks(x)
        ax1.set_xticklabels(rps_values)
        ax1.legend()
        ax1.grid(True, alpha=0.3, axis='y')

        # Set dynamic Y-axis range for CPU
        max_cpu = np.max(ig_cpu)
        ax1.set_ylim(0, max_cpu * 1.25)

        # Add value labels on CPU bars
        for i, bar in enumerate(bars1):
            height = bar.get_height()
            ci_value = baseline_cpu_ci[i]
            ax1.text(bar.get_x() + bar.get_width()/2., height + ci_value,
                     f'{height:.2f}', ha='center', va='bottom', fontsize=9, rotation=0)

        for i, bar in enumerate(bars2):
            height = bar.get_height()
            ci_value = ig_cpu_ci[i]
            ax1.text(bar.get_x() + bar.get_width()/2., height + ci_value,
                     f'{height:.2f}', ha='center', va='bottom', fontsize=9, rotation=0)

        # Plot 2: Memory Usage - simple comparison bars
        baseline_mem = baseline_data['mem(MB)'].values
        ig_mem = ig_data['mem(MB)'].values
        baseline_mem_ci = baseline_data['mem_ci'].values
        ig_mem_ci = ig_data['mem_ci'].values

        # Baseline bar
        bars3 = ax2.bar(x - width/2, baseline_mem, width, label='Baseline', alpha=0.8,
                        yerr=baseline_mem_ci, capsize=5)

        # IG Total bar
        bars4 = ax2.bar(x + width/2, ig_mem, width, label='With IG', alpha=0.8,
                        yerr=ig_mem_ci, capsize=5)

        ax2.set_xlabel('Events per Second')
        ax2.set_ylabel('Memory Usage (MB)')
        ax2.set_title('Memory Usage (with 95% CI)')
        ax2.set_xticks(x)
        ax2.set_xticklabels(rps_values)
        ax2.legend()
        ax2.grid(True, alpha=0.3, axis='y')

        # Adjust Y-axis range for memory
        all_mem_values = np.concatenate([baseline_mem, ig_mem])
        mem_min, mem_max = np.min(all_mem_values), np.max(all_mem_values)
        ax2.set_ylim(0.75*mem_min, 1.25*mem_max)

        # Add value labels on Memory bars
        for i, bar in enumerate(bars3):
            height = bar.get_height()
            ci_value = baseline_mem_ci[i]
            ax2.text(bar.get_x() + bar.get_width()/2., height + ci_value,
                     f'{height:.0f}', ha='center', va='bottom', fontsize=9, rotation=0)

        for i, bar in enumerate(bars4):
            height = bar.get_height()
            ci_value = ig_mem_ci[i]
            ax2.text(bar.get_x() + bar.get_width()/2., height + ci_value,
                     f'{height:.0f}', ha='center', va='bottom', fontsize=9, rotation=0)

        # Set main title
        fig.suptitle(f'{test} - Host', fontsize=16, fontweight='bold')

        plt.tight_layout()

        # Create IG process plots
        ig_fig = create_ig_process_plots(baseline_data, ig_data, test)

        # Create table
        df_table, styled_table = create_performance_table(baseline_data, ig_data)

        all_results.append((fig, ig_fig, aggregated_df, df_table, styled_table, test))

    return all_results

In [ ]:
# Process the configured input file
if input_file and os.path.exists(input_file):
    print(f"Processing file: {os.path.basename(input_file)}")
    try:
        results = create_performance_plots(input_file)
        if results:
            for fig, ig_fig, df, df_table, styled_table, test in results:


                # Show the system performance plot
                plt.show()

                # Show the IG process plot
                plt.show()

                # Display the styled table
                print(f"\nPerformance Metrics Table - {test}:")
                display(styled_table)
        else:
            print(f"Skipping {input_file} due to data issues")
    except Exception as e:
        print(f"Error processing {input_file}: {e}")
elif input_file:
    print(f"Error: Input file '{input_file}' not found")
else:
    print("No input file specified. Please set the 'input_file' variable in the configuration cell.")

## Raw Data

The following table shows the complete raw data from the benchmark CSV file for reference and further analysis.

In [ ]:
# Display the raw CSV data as a table
if input_file and os.path.exists(input_file):
    try:
        # Read the CSV file, ignoring lines starting with #
        raw_df = pd.read_csv(input_file, comment='#')

        # Clean column names (remove extra spaces)
        raw_df.columns = raw_df.columns.str.strip()

        # Style the raw data table and hide the index
        styled_raw_table = raw_df.style.hide(axis="index").set_table_styles([
            {'selector': 'thead th', 'props': [('background-color', '#2196F3'),
                                              ('color', 'white'),
                                              ('font-weight', 'bold'),
                                              ('text-align', 'center'),
                                              ('padding', '8px')]},
            {'selector': 'tbody td', 'props': [('text-align', 'center'),
                                               ('padding', '6px')]},
            {'selector': 'table', 'props': [('border-collapse', 'collapse'),
                                            ('margin', '25px 0'),
                                            ('font-size', '0.85em'),
                                            ('min-width', '100%'),
                                            ('border-radius', '5px 5px 0 0'),
                                            ('overflow', 'hidden'),
                                            ('box-shadow', '0 0 20px rgba(0,0,0,0.15)')]},
            {'selector': 'tbody tr:nth-of-type(even)', 'props': [('background-color', '#f3f3f3')]},
            {'selector': 'tbody tr', 'props': [('border-bottom', '1px solid #dddddd')]},
        ])

        # Display the styled table
        display(styled_raw_table)
    except Exception as e:
        print(f"Error displaying raw data: {e}")
elif input_file:
    print(f"Error: Input file '{input_file}' not found for raw data display")
else:
    print("No input file specified for raw data display.")